In [2]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [3]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_ecd_pc"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-11-14T11:28:37.079618Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-11-14T11:28:37.081106Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-11-14T11:28:37.081554Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-11-14T11:28:37.082000Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-11-14T11:28:37.082362Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4a425da-0c56-4c61-97a3-ec279e4ec8dc;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.19.0.0 in central
	found com.oracle.databa

In [4]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = b10fd2d0-6fff-4750-9e74-8ba28855899e


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [5]:
# ============================================================================
# CONFIGURAÇÃO INICIAL - 
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    concat as spark_concat,
    lit as spark_lit,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    countDistinct as spark_countDistinct
)
from pyspark.sql.types import DoubleType, IntegerType

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# ✅ CORREÇÃO: Não usar abs() que conflita com PySpark
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}' if abs(x) > 0.01 else f'{x:.6f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Acesso ao Spark
spark = session.sparkSession

print("=" * 80)
print("🔍 SISTEMA")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

🔍 SISTEMA
Sessão Spark: tsevero_ecd_pc
Versão Spark: 3.5.4.3.5.7191000.0-30
Iniciado em: 2025-11-14 08:28:56


In [6]:
# ============================================================================
# CONFIGURAÇÕES
# ============================================================================

# Parâmetros
ANO_REFERENCIA = 2024
TOLERANCIA_EQUACAO_CONTABIL = 1000.0  # R$ 1.000
LIMITE_CONTAS = None

# Tabelas
SCHEMA = "usr_sat_ecd"
TABLE_PLANO_CONTAS = f"{SCHEMA}.ecd_ri050_plano_contas"
TABLE_SALDOS = f"{SCHEMA}.ecd_ri155_detalhe_saldos_periodicos"
TABLE_BP = f"{SCHEMA}.ecd_rj100_balanco_patrimonial"
TABLE_DRE = f"{SCHEMA}.ecd_rj150_demonstracao_resultado_exercicio"
TABLE_IDENTIFICACAO = f"{SCHEMA}.ecd_r0000_identificacao"
TABLE_REF_BP = "teste.ecd_pc_bp"
TABLE_REF_DRE = "teste.ecd_pc_dre"

print(f"✅ Configurações definidas - Ano: {ANO_REFERENCIA}")

✅ Configurações definidas - Ano: 2024


In [7]:
# ============================================================================
# EXPLORAÇÃO: Identificar empresas com BP e DRE já classificados
# ============================================================================

print("🔍 Identificando empresas com dados rotulados...")

# Empresas com BP
df_bp_empresas = spark.sql(f"""
    SELECT DISTINCT id_ecd, dt_referencia
    FROM {TABLE_BP}
    WHERE dt_referencia >= {ANO_REFERENCIA}
""")

# Empresas com DRE
df_dre_empresas = spark.sql(f"""
    SELECT DISTINCT id_ecd, dt_referencia
    FROM {TABLE_DRE}
    WHERE dt_referencia >= {ANO_REFERENCIA}
""")

# Empresas completas (com ambos)
df_empresas_rotuladas = df_bp_empresas.join(
    df_dre_empresas, on=['id_ecd', 'dt_referencia'], how='inner'
)

# Adicionar informações cadastrais
df_empresas_info = spark.sql(f"""
    SELECT id_ecd, dt_referencia, nm_empresarial, nu_cnpj, cd_uf, ind_grande_porte
    FROM {TABLE_IDENTIFICACAO}
    WHERE dt_referencia >= {ANO_REFERENCIA}
""")

df_empresas_rotuladas = df_empresas_rotuladas.join(
    df_empresas_info, on=['id_ecd', 'dt_referencia'], how='inner'
)

total_rotuladas = df_empresas_rotuladas.count()
print(f"✅ {total_rotuladas:,} empresas com dados rotulados")

df_empresas_rotuladas.show(5, truncate=False)

🔍 Identificando empresas com dados rotulados...


✅ 230,487 empresas com dados rotulados


[Stage 13:============>   (22 + 7) / 29][Stage 14:===========>      (5 + 3) / 8]

+------+-------------+--------------------------------------------+--------------+-----+----------------+
|id_ecd|dt_referencia|nm_empresarial                              |nu_cnpj       |cd_uf|ind_grande_porte|
+------+-------------+--------------------------------------------+--------------+-----+----------------+
|143   |202101       |NIVIA INDUSTRIA E COMERCIO DE ALIMENTOS LTDA|11389668000199|SC   |0               |
|905   |202001       |RIBEIRO E FREITAS TRANSPORTES LTDA ME       |14462133000194|SC   |0               |
|928   |202101       |EASY TRADING IMPORTAÇÃO E EXPORTAÇÃO LTDA   |11580254000142|SC   |0               |
|1095  |202001       |ABATEDOURO MARAVILHA EIRELI                 |12394521000150|SC   |0               |
|1105  |201901       |CM INDUSTRIA DE CALCADOS LTDA               |81019895000108|SC   |0               |
+------+-------------+--------------------------------------------+--------------+-----+----------------+
only showing top 5 rows



In [8]:
# ============================================================================
# ANÁLISES ESTATÍSTICAS
# ============================================================================

print("📊 Analisando distribuição de naturezas...")

df_naturezas = spark.sql(f"""
    SELECT 
        cd_natureza,
        CASE cd_natureza
            WHEN '01' THEN 'Devedora'
            WHEN '02' THEN 'Credora'
            WHEN '03' THEN 'Híbrida'
            WHEN '04' THEN 'Resultado'
            ELSE 'Outros'
        END as natureza_desc,
        COUNT(DISTINCT id_ecd) as qtd_empresas,
        COUNT(*) as qtd_contas
    FROM {TABLE_PLANO_CONTAS}
    WHERE dt_referencia >= {ANO_REFERENCIA}
    GROUP BY cd_natureza
    ORDER BY qtd_contas DESC
""")

df_naturezas.show(truncate=False)

# Níveis hierárquicos
print("\n📊 Analisando níveis hierárquicos...")

df_niveis = spark.sql(f"""
    SELECT nivel, COUNT(DISTINCT id_ecd) as qtd_empresas, COUNT(*) as qtd_contas
    FROM {TABLE_PLANO_CONTAS}
    WHERE dt_referencia >= {ANO_REFERENCIA}
    GROUP BY nivel
    ORDER BY nivel
""")

df_niveis.show(15, truncate=False)

# Padrões de códigos
print("\n📊 Padrões de códigos (primeiro caractere)...")

spark.sql(f"""
    SELECT SUBSTRING(cd_conta_anl, 1, 1) as primeiro_char, cd_natureza, COUNT(*) as total
    FROM {TABLE_PLANO_CONTAS}
    WHERE dt_referencia >= {ANO_REFERENCIA}
    GROUP BY SUBSTRING(cd_conta_anl, 1, 1), cd_natureza
    ORDER BY total DESC
    LIMIT 15
""").show(truncate=False)

📊 Analisando distribuição de naturezas...


+-----------+-------------+------------+----------+
|cd_natureza|natureza_desc|qtd_empresas|qtd_contas|
+-----------+-------------+------------+----------+
|01         |Devedora     |418889      |221642840 |
|04         |Resultado    |419064      |142007019 |
|02         |Credora      |418960      |133530262 |
|03         |Híbrida      |364296      |6495314   |
|05         |Outros       |88360       |2473512   |
|09         |Outros       |237619      |2304241   |
+-----------+-------------+------------+----------+


📊 Analisando níveis hierárquicos...


+-----+------------+----------+
|nivel|qtd_empresas|qtd_contas|
+-----+------------+----------+
|1    |419128      |2120936   |
|2    |419113      |4684632   |
|3    |419112      |12610928  |
|4    |419109      |88752168  |
|5    |413729      |304583723 |
|6    |204501      |75316968  |
|7    |57041       |13213348  |
|8    |25576       |5182606   |
|9    |19429       |1475313   |
|10   |13616       |391786    |
|11   |4592        |117747    |
|12   |318         |3033      |
+-----+------------+----------+


📊 Padrões de códigos (primeiro caractere)...


[Stage 31:====================================================> (106 + 3) / 109]

+-------------+-----------+---------+
|primeiro_char|cd_natureza|total    |
+-------------+-----------+---------+
|1            |01         |123523368|
|2            |02         |80085516 |
|0            |01         |38984149 |
|0            |04         |33899116 |
|B            |04         |28400628 |
|3            |04         |26789810 |
|0            |02         |22459224 |
|B            |01         |17361387 |
|4            |04         |16192858 |
|5            |04         |12507357 |
|9            |01         |10717718 |
|1            |02         |10136781 |
|2            |01         |7198508  |
|6            |04         |6329043  |
|3            |01         |5191601  |
+-------------+-----------+---------+



In [10]:
# ============================================================================
# CARREGAR PLANO DE CONTAS PARA CLASSIFICAÇÃO (v2 - COM NOMES CORRETOS)
# ============================================================================
from pyspark.sql import functions as F

print("📥 Carregando plano de contas COM NOMES CORRETOS (BP/DRE)...")

# Verificar se tabela v2 existe
try:
    test_v2 = spark.sql("SELECT COUNT(*) as total FROM teste.ecd_contas_classificadas_v2").collect()[0]['total']
    print(f"✅ Usando ecd_contas_classificadas_v2: {test_v2:,} registros disponíveis")
    usar_v2 = True
except:
    print("⚠️  Tabela v2 não encontrada, usando tabela original")
    usar_v2 = False

if usar_v2:
    # ✅ VERSÃO NOVA: Usar ecd_contas_classificadas_v2 (com nomes corretos de BP/DRE)
    query = f"""
        SELECT 
            id_ecd,
            ano_referencia AS dt_referencia,
            cd_conta AS cd_conta_anl,
            cd_conta_sint,
            nm_conta AS nm_conta_anl,  -- ✅ NOME CORRETO DE BP/DRE
            cd_natureza,
            nivel,
            tp_conta,
            origem,  -- Extra: BP ou DRE
            classificacao_final,  -- Extra: classificação existente
            confianca  -- Extra: nível de confiança
        FROM teste.ecd_contas_classificadas_v2
        WHERE CAST(ano_referencia / 100 AS INT) >= {ANO_REFERENCIA}
          AND tp_conta = 'A'  -- Apenas contas analíticas
          AND nm_conta IS NOT NULL
          AND nm_conta != 'CONTA ANALÍTICA'
          AND nm_conta != ''
    """
    
    if LIMITE_CONTAS:
        query += f" LIMIT {LIMITE_CONTAS}"
    
    df_plano = spark.sql(query)
    total = df_plano.count()
    
    print(f"✅ {total:,} contas carregadas COM NOMES CORRETOS")
    print(f"📊 Fonte dos nomes: Balanço Patrimonial (BP) e DRE")
    
    # Estatísticas de qualidade
    print("\n📊 Estatísticas de qualidade dos dados:")
    df_plano.groupBy('origem').count().orderBy('origem').show()
    
    print("📊 Distribuição por confiança:")
    df_plano.groupBy('confianca').count().orderBy('confianca').show()
    
    print("\n📋 Amostra de dados:")
    df_plano.select(
        'id_ecd', 'dt_referencia', 'cd_conta_anl', 'nm_conta_anl', 
        'origem', 'confianca'
    ).show(5, truncate=False)

else:
    # VERSÃO ANTIGA: Fallback para tabela original
    query = f"""
        SELECT 
            id_ecd, 
            dt_referencia, 
            cd_conta_anl, 
            cd_conta_sint,
            nm_conta_anl, 
            cd_natureza, 
            nivel, 
            tp_conta
        FROM {TABLE_PLANO_CONTAS}
        WHERE dt_referencia >= {ANO_REFERENCIA}
          AND tp_conta = 'A'  -- Apenas contas analíticas
    """
    
    if LIMITE_CONTAS:
        query += f" LIMIT {LIMITE_CONTAS}"
    
    df_plano = spark.sql(query)
    total = df_plano.count()
    
    print(f"✅ {total:,} contas carregadas (fallback - tabela original)")
    print("⚠️  ATENÇÃO: Nomes podem estar genéricos ('CONTA ANALÍTICA')")
    
    print("\n📋 Amostra de dados:")
    df_plano.show(5, truncate=False)

# Validação final
print(f"\n✅ Total de contas no DataFrame: {df_plano.count():,}")
print(f"✅ Colunas disponíveis: {df_plano.columns}")

📥 Carregando plano de contas COM NOMES CORRETOS (BP/DRE)...


✅ Usando ecd_contas_classificadas_v2: 120,989,084 registros disponíveis


✅ 18,925,701 contas carregadas COM NOMES CORRETOS
📊 Fonte dos nomes: Balanço Patrimonial (BP) e DRE

📊 Estatísticas de qualidade dos dados:


+------+--------+
|origem|   count|
+------+--------+
|    BP|13955691|
|   DRE| 4970010|
+------+--------+

📊 Distribuição por confiança:


+---------+--------+
|confianca|   count|
+---------+--------+
|     ALTA| 3070040|
|    BAIXA|10980692|
|    MEDIA| 4874969|
+---------+--------+


📋 Amostra de dados:
+------+-------------+------------+--------------------------------+------+---------+
|id_ecd|dt_referencia|cd_conta_anl|nm_conta_anl                    |origem|confianca|
+------+-------------+------------+--------------------------------+------+---------+
|346788|202401       |480         |(-) SIMPLES NACIONAL            |DRE   |MEDIA    |
|346791|202401       |268         |RESULTADO DO EXERCÍCIO EM CURSO |BP    |MEDIA    |
|346792|202401       |165         |FORNECEDORES                    |BP    |BAIXA    |
|346802|202401       |362         |SERVIÇOS PRESTADOS POR TERCEIROS|DRE   |ALTA     |
|346804|202401       |55          |MERCADORIAS PARA REVENDA        |BP    |BAIXA    |
+------+-------------+------------+--------------------------------+------+---------+
only showing top 5 rows



[Stage 57:================================================>  (1145 + 28) / 1200]


✅ Total de contas no DataFrame: 18,925,701
✅ Colunas disponíveis: ['id_ecd', 'dt_referencia', 'cd_conta_anl', 'cd_conta_sint', 'nm_conta_anl', 'cd_natureza', 'nivel', 'tp_conta', 'origem', 'classificacao_final', 'confianca']


In [11]:
# ============================================================================
# FEATURE ENGINEERING - FEATURES DO CÓDIGO
# ============================================================================

print("⚙️ Extraindo features do código...")

df_features = df_plano \
    .withColumn('primeiro_digito', regexp_extract(col('cd_conta_anl'), r'^(\d)', 1)) \
    .withColumn('prefixo_2_chars', upper(substring(col('cd_conta_anl'), 1, 2))) \
    .withColumn('tem_ponto', when(col('cd_conta_anl').contains('.'), 1).otherwise(0)) \
    .withColumn('comprimento_codigo', length(col('cd_conta_anl'))) \
    .withColumn('qtd_pontos', length(col('cd_conta_anl')) - length(regexp_replace(col('cd_conta_anl'), r'\.', ''))) \
    .withColumn('qtd_digitos', length(regexp_replace(col('cd_conta_anl'), r'[^\d]', ''))) \
    .withColumn('tem_letra', when(col('cd_conta_anl').rlike('[a-zA-Z]'), 1).otherwise(0)) \
    .withColumn('padrao_estrutural',
        when(col('cd_conta_anl').rlike(r'^\d+\.\d+\.\d+'), 'PADRAO_PONTOS')
        .when(col('cd_conta_anl').rlike(r'^BP'), 'PREFIXO_BP')
        .when(col('cd_conta_anl').rlike(r'^DRE'), 'PREFIXO_DRE')
        .when(col('cd_conta_anl').rlike(r'^\d+$'), 'SOMENTE_NUMEROS')
        .otherwise('OUTRO')
    )

print("✅ Features extraídas")

print("\n📊 Distribuição de padrões:")
df_features.groupBy('padrao_estrutural').count().orderBy(desc('count')).show(10)

⚙️ Extraindo features do código...
✅ Features extraídas

📊 Distribuição de padrões:


[Stage 60:==================================================>(1179 + 21) / 1200]

+-----------------+-------+
|padrao_estrutural|  count|
+-----------------+-------+
|  SOMENTE_NUMEROS|9618668|
|       PREFIXO_BP|4944693|
|    PADRAO_PONTOS|2345098|
|      PREFIXO_DRE|1087728|
|            OUTRO| 929514|
+-----------------+-------+



In [12]:
# ============================================================================
# FEATURE ENGINEERING - NATUREZA (ONE-HOT ENCODING)
# ============================================================================

print("⚙️ Features de natureza...")

df_features = df_features \
    .withColumn('natureza_devedora', when(col('cd_natureza') == '01', 1).otherwise(0)) \
    .withColumn('natureza_credora', when(col('cd_natureza') == '02', 1).otherwise(0)) \
    .withColumn('natureza_resultado', when(col('cd_natureza') == '04', 1).otherwise(0))

print("✅ Features de natureza criadas")

df_features.groupBy('cd_natureza').agg(
    sum('natureza_devedora').alias('total_devedora'),
    sum('natureza_credora').alias('total_credora'),
    sum('natureza_resultado').alias('total_resultado')
).show()

⚙️ Features de natureza...
✅ Features de natureza criadas


[Stage 63:================================================>  (1145 + 28) / 1200]

+-----------+--------------+-------------+---------------+
|cd_natureza|total_devedora|total_credora|total_resultado|
+-----------+--------------+-------------+---------------+
|          D|             0|            0|              0|
|          C|             0|            0|              0|
+-----------+--------------+-------------+---------------+



In [13]:
# ============================================================================
# FEATURE ENGINEERING - SIMILARIDADE COM REFERÊNCIAS
# ============================================================================

print("⚙️ Features de similaridade...")

# Carregar referências
df_ref_bp = spark.table(TABLE_REF_BP).select(col('codigo').alias('ref_bp')).distinct()
df_ref_dre = spark.table(TABLE_REF_DRE).select(col('codigo').alias('ref_dre')).distinct()

print(f"   Ref BP: {df_ref_bp.count()} | Ref DRE: {df_ref_dre.count()}")

# Match exato
df_features = df_features \
    .join(df_ref_bp, col('cd_conta_anl') == col('ref_bp'), 'left') \
    .withColumn('match_exato_bp', when(col('ref_bp').isNotNull(), 1).otherwise(0)) \
    .drop('ref_bp') \
    .join(df_ref_dre, col('cd_conta_anl') == col('ref_dre'), 'left') \
    .withColumn('match_exato_dre', when(col('ref_dre').isNotNull(), 1).otherwise(0)) \
    .drop('ref_dre')

# Match parcial (por padrão de código)
df_features = df_features \
    .withColumn('match_parcial_ativo',
        when(col('primeiro_digito') == '1', 1)
        .when(col('prefixo_2_chars') == '01', 1)
        .otherwise(0)
    ) \
    .withColumn('match_parcial_passivo',
        when(col('primeiro_digito') == '2', 1)
        .when(col('prefixo_2_chars') == '02', 1)
        .otherwise(0)
    ) \
    .withColumn('match_parcial_dre',
        when(col('primeiro_digito') == '3', 1)
        .when(col('prefixo_2_chars') == '03', 1)
        .otherwise(0)
    )

# Grupo provável
df_features = df_features.withColumn('grupo_provavel',
    when((col('cd_natureza') == '01') & (col('nivel') == 1), 'ATIVO')
    .when((col('cd_natureza') == '02') & (col('nivel') <= 2), 'PASSIVO_PL')
    .when(col('cd_natureza') == '04', 'DRE')
    .when(col('match_parcial_ativo') == 1, 'ATIVO')
    .when(col('match_parcial_passivo') == 1, 'PASSIVO_PL')
    .when(col('match_parcial_dre') == 1, 'DRE')
    .otherwise('INDEFINIDO')
)

print("✅ Features de similaridade criadas")

df_features.groupBy('grupo_provavel').count().orderBy(desc('count')).show()

⚙️ Features de similaridade...
   Ref BP: 722 | Ref DRE: 387
✅ Features de similaridade criadas


[Stage 78:=================================================> (1155 + 28) / 1200]

+--------------+--------+
|grupo_provavel|   count|
+--------------+--------+
|    INDEFINIDO|11299279|
|         ATIVO| 3794535|
|    PASSIVO_PL| 2360285|
|           DRE| 1471602|
+--------------+--------+



In [14]:
# ============================================================================
# CLASSIFICAÇÃO POR REGRAS - NÍVEL 1 (GRUPOS PRINCIPAIS)
# ============================================================================

print("📋 Aplicando regras de Nível 1...")

df_classificado = df_features.withColumn('classificacao_nivel_1',
    when((col('cd_natureza') == '01') & (col('nivel') == 1), '1')
    .when((col('cd_natureza') == '02') & (col('nivel') == 1), '2')
    .when(col('cd_natureza') == '04', '3')
    .when(col('primeiro_digito') == '1', '1')
    .when(col('prefixo_2_chars') == '01', '1')
    .when(col('primeiro_digito') == '2', '2')
    .when(col('prefixo_2_chars') == '02', '2')
    .when(col('primeiro_digito') == '3', '3')
    .when(col('prefixo_2_chars') == '03', '3')
    .otherwise(None)
).withColumn('classificacao_nivel_1_desc',
    when(col('classificacao_nivel_1') == '1', 'ATIVO')
    .when(col('classificacao_nivel_1') == '2', 'PASSIVO')
    .when(col('classificacao_nivel_1') == '3', 'RESULTADO')
    .otherwise(None)
).withColumn('confianca_nivel_1',
    when((col('cd_natureza') == '01') & (col('nivel') == 1), 0.95)
    .when((col('cd_natureza') == '02') & (col('nivel') == 1), 0.95)
    .when(col('cd_natureza') == '04', 0.90)
    .when(col('primeiro_digito').isin(['1', '2', '3']), 0.80)
    .otherwise(0.0)
)

total = df_classificado.count()
classificados = df_classificado.filter(col('classificacao_nivel_1').isNotNull()).count()

print(f"✅ Nível 1: {classificados:,} / {total:,} ({classificados/total*100:.1f}%)")

df_classificado.groupBy('classificacao_nivel_1_desc').count().show()

📋 Aplicando regras de Nível 1...


✅ Nível 1: 7,626,422 / 18,925,701 (40.3%)


[Stage 87:=================================================> (1168 + 28) / 1200]

+--------------------------+--------+
|classificacao_nivel_1_desc|   count|
+--------------------------+--------+
|                      NULL|11299279|
|                   PASSIVO| 2360285|
|                 RESULTADO| 1471602|
|                     ATIVO| 3794535|
+--------------------------+--------+



In [15]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 2: SUBGRUPOS DO ATIVO
# ============================================================================

print("📋 Aplicando regras de Nível 2 - ATIVO...")

df_classificado = df_classificado.withColumn('classificacao_nivel_2',
    # Ativo Circulante (1.01)
    when(
        (col('classificacao_nivel_1') == '1') & 
        (col('cd_conta_anl').rlike(r'^1\.?01') | col('cd_conta_anl').rlike(r'^01\.?1\.?01')),
        '1.01'
    )
    # Ativo Não Circulante (1.02)
    .when(
        (col('classificacao_nivel_1') == '1') & 
        (col('cd_conta_anl').rlike(r'^1\.?02') | col('cd_conta_anl').rlike(r'^01\.?1\.?02')),
        '1.02'
    )
    # Ativo Permanente/Investimentos (1.03)
    .when(
        (col('classificacao_nivel_1') == '1') & 
        (col('cd_conta_anl').rlike(r'^1\.?03') | col('cd_conta_anl').rlike(r'^01\.?1\.?03')),
        '1.03'
    )
    .otherwise(col('classificacao_nivel_1'))
)

print("✅ Regras Ativo aplicadas")

df_classificado.filter(col('classificacao_nivel_1') == '1') \
    .groupBy('classificacao_nivel_2').count().show()

📋 Aplicando regras de Nível 2 - ATIVO...
✅ Regras Ativo aplicadas


[Stage 90:==================================================>(1181 + 19) / 1200]

+---------------------+-------+
|classificacao_nivel_2|  count|
+---------------------+-------+
|                 1.01| 214745|
|                 1.02|  89814|
|                 1.03|  53312|
|                    1|3436664|
+---------------------+-------+



In [16]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 2: SUBGRUPOS DO PASSIVO E PL
# ============================================================================

print("📋 Aplicando regras de Nível 2 - PASSIVO e PL...")

df_classificado = df_classificado.withColumn('classificacao_nivel_2',
    # Passivo Circulante (2.01)
    when(
        (col('classificacao_nivel_1') == '2') & 
        (col('cd_conta_anl').rlike(r'^2\.?01') | col('cd_conta_anl').rlike(r'^02\.?1\.?01')),
        '2.01'
    )
    # Passivo Não Circulante (2.02)
    .when(
        (col('classificacao_nivel_1') == '2') & 
        (col('cd_conta_anl').rlike(r'^2\.?02') | col('cd_conta_anl').rlike(r'^02\.?1\.?02')),
        '2.02'
    )
    # Patrimônio Líquido (2.03)
    .when(
        (col('classificacao_nivel_1') == '2') & 
        (col('cd_conta_anl').rlike(r'^2\.?03') | col('cd_conta_anl').rlike(r'^02\.?3')),
        '2.03'
    )
    .otherwise(col('classificacao_nivel_2'))
)

print("✅ Regras Passivo/PL aplicadas")

df_classificado.filter(col('classificacao_nivel_1') == '2') \
    .groupBy('classificacao_nivel_2').count().orderBy(desc('count')).show()

📋 Aplicando regras de Nível 2 - PASSIVO e PL...
✅ Regras Passivo/PL aplicadas


[Stage 93:===================================================>(1197 + 3) / 1200]

+---------------------+-------+
|classificacao_nivel_2|  count|
+---------------------+-------+
|                    2|2163258|
|                 2.01| 134358|
|                 2.02|  38293|
|                 2.03|  24376|
+---------------------+-------+



In [17]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 2: SUBGRUPOS DA DRE
# ============================================================================

print("📋 Aplicando regras de Nível 2 - DRE...")

df_classificado = df_classificado.withColumn('classificacao_nivel_2',
    # Receita Operacional (3.01)
    when(
        (col('classificacao_nivel_1') == '3') & 
        (col('cd_conta_anl').rlike(r'^3\.?01') | col('cd_conta_anl').rlike(r'^03\.?01')),
        '3.01'
    )
    # Custo (3.02)
    .when(
        (col('classificacao_nivel_1') == '3') & 
        (col('cd_conta_anl').rlike(r'^3\.?02') | col('cd_conta_anl').rlike(r'^03\.?02')),
        '3.02'
    )
    # Despesa (3.03)
    .when(
        (col('classificacao_nivel_1') == '3') & 
        (col('cd_conta_anl').rlike(r'^3\.?03') | col('cd_conta_anl').rlike(r'^03\.?03')),
        '3.03'
    )
    .otherwise(col('classificacao_nivel_2'))
)

print("✅ Regras DRE aplicadas")

df_classificado.filter(col('classificacao_nivel_1') == '3') \
    .groupBy('classificacao_nivel_2').count().orderBy(desc('count')).show()

📋 Aplicando regras de Nível 2 - DRE...
✅ Regras DRE aplicadas


[Stage 96:=================================================> (1153 + 28) / 1200]

+---------------------+-------+
|classificacao_nivel_2|  count|
+---------------------+-------+
|                    3|1387603|
|                 3.01|  52079|
|                 3.02|  16592|
|                 3.03|  15328|
+---------------------+-------+



In [18]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 3: DETALHAMENTO DO ATIVO
# ============================================================================

print("📋 Aplicando regras de Nível 3 - Detalhamento Ativo...")

df_classificado = df_classificado.withColumn('classificacao_nivel_3',
    # Disponibilidades (1.01.01)
    when(
        (col('classificacao_nivel_2') == '1.01') & 
        (lower(col('cd_conta_anl')).rlike('caixa|banco|disponib|numerario|dinheiro|vista')),
        '1.01.01'
    )
    # Aplicações Financeiras (1.01.02)
    .when(
        (col('classificacao_nivel_2') == '1.01') & 
        (lower(col('cd_conta_anl')).rlike('aplicacao|investimento.*curto|fundo.*investimento|cdb')),
        '1.01.02'
    )
    # Clientes/Duplicatas a Receber (1.01.03)
    .when(
        (col('classificacao_nivel_2') == '1.01') & 
        (lower(col('cd_conta_anl')).rlike('cliente|duplicata.*receber|contas.*receber|recebivel')),
        '1.01.03'
    )
    # Estoques (1.01.04)
    .when(
        (col('classificacao_nivel_2') == '1.01') & 
        (lower(col('cd_conta_anl')).rlike('estoque|mercadoria|produto|materia.*prima|almoxarifado')),
        '1.01.04'
    )
    # Tributos a Recuperar (1.01.05)
    .when(
        (col('classificacao_nivel_2') == '1.01') & 
        (lower(col('cd_conta_anl')).rlike('tributo.*recuperar|imposto.*recuperar|icms.*recuperar|iva|pis|cofins|ipi')),
        '1.01.05'
    )
    .otherwise(col('classificacao_nivel_2'))
)

print("✅ Regras Nível 3 Ativo aplicadas")

df_classificado.filter(col('classificacao_nivel_2') == '1.01') \
    .groupBy('classificacao_nivel_3').count().orderBy(desc('count')).show()

📋 Aplicando regras de Nível 3 - Detalhamento Ativo...
✅ Regras Nível 3 Ativo aplicadas


[Stage 99:===================================================>(1198 + 2) / 1200]

+---------------------+------+
|classificacao_nivel_3| count|
+---------------------+------+
|                 1.01|214745|
+---------------------+------+



In [19]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 3: DETALHAMENTO DO PASSIVO
# ============================================================================

print("📋 Aplicando regras de Nível 3 - Detalhamento Passivo...")

df_classificado = df_classificado.withColumn('classificacao_nivel_3',
    # Fornecedores (2.01.01)
    when(
        (col('classificacao_nivel_2') == '2.01') & 
        (lower(col('cd_conta_anl')).rlike('fornecedor|duplicata.*pagar|contas.*pagar|obrigacoes|pagavel')),
        '2.01.01'
    )
    # Empréstimos e Financiamentos CP (2.01.02)
    .when(
        (col('classificacao_nivel_2') == '2.01') & 
        (lower(col('cd_conta_anl')).rlike('emprestimo|financiamento|banco.*curto|divida.*curto')),
        '2.01.02'
    )
    # Tributos a Recolher (2.01.03)
    .when(
        (col('classificacao_nivel_2') == '2.01') & 
        (lower(col('cd_conta_anl')).rlike('tributo.*pagar|tributo.*recolher|imposto.*pagar|icms.*recolher|irpj|csll')),
        '2.01.03'
    )
    # Salários e Encargos (2.01.04)
    .when(
        (col('classificacao_nivel_2') == '2.01') & 
        (lower(col('cd_conta_anl')).rlike('salario|folha.*pagamento|fgts|inss|encargo|trabalhista')),
        '2.01.04'
    )
    .otherwise(col('classificacao_nivel_3'))
)

print("✅ Regras Nível 3 Passivo aplicadas")

df_classificado.filter(col('classificacao_nivel_2') == '2.01') \
    .groupBy('classificacao_nivel_3').count().orderBy(desc('count')).show()

📋 Aplicando regras de Nível 3 - Detalhamento Passivo...
✅ Regras Nível 3 Passivo aplicadas


[Stage 102:==================================================>(1197 + 3) / 1200]

+---------------------+------+
|classificacao_nivel_3| count|
+---------------------+------+
|                 2.01|134358|
+---------------------+------+



In [20]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 3: DETALHAMENTO DO PATRIMÔNIO LÍQUIDO
# ============================================================================

print("📋 Aplicando regras de Nível 3 - Detalhamento PL...")

df_classificado = df_classificado.withColumn('classificacao_nivel_3',
    # Capital Social (2.03.01)
    when(
        (col('classificacao_nivel_2') == '2.03') & 
        (lower(col('cd_conta_anl')).rlike('capital.*social|capital.*subscrito|capital.*integralizado')),
        '2.03.01'
    )
    # Reservas (2.03.02)
    .when(
        (col('classificacao_nivel_2') == '2.03') & 
        (lower(col('cd_conta_anl')).rlike('reserva|reserva.*capital|reserva.*lucro')),
        '2.03.02'
    )
    # Lucros/Prejuízos Acumulados (2.03.03)
    .when(
        (col('classificacao_nivel_2') == '2.03') & 
        (lower(col('cd_conta_anl')).rlike('lucro.*acumulado|prejuizo.*acumulado|resultado.*acumulado')),
        '2.03.03'
    )
    .otherwise(col('classificacao_nivel_3'))
)

print("✅ Regras Nível 3 PL aplicadas")

df_classificado.filter(col('classificacao_nivel_2') == '2.03') \
    .groupBy('classificacao_nivel_3').count().orderBy(desc('count')).show()

📋 Aplicando regras de Nível 3 - Detalhamento PL...
✅ Regras Nível 3 PL aplicadas


[Stage 105:==================================================>(1197 + 3) / 1200]

+---------------------+-----+
|classificacao_nivel_3|count|
+---------------------+-----+
|                 2.03|24376|
+---------------------+-----+



In [21]:
# ============================================================================
# CLASSIFICAÇÃO - NÍVEL 3: DETALHAMENTO DA DRE
# ============================================================================

print("📋 Aplicando regras de Nível 3 - Detalhamento DRE...")

df_classificado = df_classificado.withColumn('classificacao_nivel_3',
    # Receita Bruta (3.01.01)
    when(
        (col('classificacao_nivel_2') == '3.01') & 
        (lower(col('cd_conta_anl')).rlike('receita.*bruta|receita.*venda|receita.*servico|faturamento')),
        '3.01.01'
    )
    # Deduções da Receita (3.01.02)
    .when(
        (col('classificacao_nivel_2') == '3.01') & 
        (lower(col('cd_conta_anl')).rlike('deducao|devolucao|desconto|abatimento|icms.*venda')),
        '3.01.02'
    )
    # CMV/CPV (3.02.01)
    .when(
        (col('classificacao_nivel_2') == '3.02') & 
        (lower(col('cd_conta_anl')).rlike('custo.*mercadoria|custo.*produto|custo.*servico|cmv|cpv|csv')),
        '3.02.01'
    )
    # Despesas Operacionais (3.03.01)
    .when(
        (col('classificacao_nivel_2') == '3.03') & 
        (lower(col('cd_conta_anl')).rlike('despesa.*administrativa|despesa.*comercial|despesa.*vendas|despesa.*operacional')),
        '3.03.01'
    )
    # Despesas Financeiras (3.03.02)
    .when(
        (col('classificacao_nivel_2') == '3.03') & 
        (lower(col('cd_conta_anl')).rlike('despesa.*financeira|juros.*passivo|encargo.*financeiro')),
        '3.03.02'
    )
    # Receitas Financeiras (3.01.03)
    .when(
        (col('classificacao_nivel_2') == '3.01') & 
        (lower(col('cd_conta_anl')).rlike('receita.*financeira|juros.*ativo|rendimento.*aplicacao')),
        '3.01.03'
    )
    .otherwise(col('classificacao_nivel_3'))
)

print("✅ Regras Nível 3 DRE aplicadas")

df_classificado.filter(col('classificacao_nivel_1') == '3') \
    .groupBy('classificacao_nivel_3').count().orderBy(desc('count')).show(20)

📋 Aplicando regras de Nível 3 - Detalhamento DRE...
✅ Regras Nível 3 DRE aplicadas


[Stage 108:===============================================>  (1139 + 28) / 1200]

+---------------------+-------+
|classificacao_nivel_3|  count|
+---------------------+-------+
|                    3|1387603|
|                 3.01|  52079|
|                 3.02|  16592|
|                 3.03|  15328|
+---------------------+-------+



In [22]:
from pyspark.sql.functions import col, count, sum as _sum, when

print("📊 CALCULANDO ESTATÍSTICAS (MODO OTIMIZADO)...\n")

# Crie as colunas de flag em uma única passagem
df_stats = df_classificado.select(
    (col('classificacao_nivel_1').isNotNull()).alias('is_n1'),
    (
        (col('classificacao_nivel_2').isNotNull()) &
        (col('classificacao_nivel_2') != col('classificacao_nivel_1'))
    ).alias('is_n2'),
    (
        (col('classificacao_nivel_3').isNotNull()) &
        (col('classificacao_nivel_3') != col('classificacao_nivel_2'))
    ).alias('is_n3')
)

# Calcule todas as contagens de uma vez só
stats = df_stats.select(
    count('*').alias('total'),
    _sum(when(col('is_n1'), 1).otherwise(0)).alias('n1_class'),
    _sum(when(col('is_n2'), 1).otherwise(0)).alias('n2_class'),
    _sum(when(col('is_n3'), 1).otherwise(0)).alias('n3_class')
).first() # .first() coleta o resultado para o driver

# Agora use os resultados
total = stats['total']
n1_class = stats['n1_class']
n2_class = stats['n2_class']
n3_class = stats['n3_class']

print(f"Nível 1: {n1_class:,} / {total:,} ({n1_class/total*100:.1f}%)")
print(f"Nível 2: {n2_class:,} / {n1_class:,} ({n2_class/n1_class*100:.1f}%)")
print(f"Nível 3: {n3_class:,} / {n2_class:,} ({n3_class/n2_class*100:.1f}% dos N2)")

print("\n--- Distribuição Final por Grupo ---")
df_classificado.groupBy('classificacao_nivel_1_desc').count() \
    .orderBy(desc('count')).show()

📊 CALCULANDO ESTATÍSTICAS (MODO OTIMIZADO)...



Nível 1: 7,626,422 / 18,925,701 (40.3%)
Nível 2: 638,897 / 7,626,422 (8.4%)
Nível 3: 0 / 638,897 (0.0% dos N2)

--- Distribuição Final por Grupo ---


[Stage 114:================================================> (1153 + 28) / 1200]

+--------------------------+--------+
|classificacao_nivel_1_desc|   count|
+--------------------------+--------+
|                      NULL|11299279|
|                     ATIVO| 3794535|
|                   PASSIVO| 2360285|
|                 RESULTADO| 1471602|
+--------------------------+--------+



In [23]:
# ============================================================================
# CONSOLIDAR CLASSIFICAÇÃO FINAL
# ============================================================================

print("🔄 Consolidando classificação final...")

# Criar coluna de classificação final (mais detalhada disponível)
df_classificado = df_classificado.withColumn('classificacao_final',
    coalesce(
        col('classificacao_nivel_3'),
        col('classificacao_nivel_2'),
        col('classificacao_nivel_1')
    )
).withColumn('nivel_classificacao',
    when(col('classificacao_nivel_3') != col('classificacao_nivel_2'), 3)
    .when(col('classificacao_nivel_2') != col('classificacao_nivel_1'), 2)
    .when(col('classificacao_nivel_1').isNotNull(), 1)
    .otherwise(0)
)

print("✅ Classificação consolidada")

print("\n--- Distribuição por Nível de Detalhamento ---")
df_classificado.groupBy('nivel_classificacao').count() \
    .orderBy('nivel_classificacao').show()

print("\n--- Top 20 Classificações Finais ---")
df_classificado.groupBy('classificacao_final').count() \
    .orderBy(desc('count')).show(20)

🔄 Consolidando classificação final...
✅ Classificação consolidada

--- Distribuição por Nível de Detalhamento ---


+-------------------+--------+
|nivel_classificacao|   count|
+-------------------+--------+
|                  0|11299279|
|                  1| 6987525|
|                  2|  638897|
+-------------------+--------+


--- Top 20 Classificações Finais ---


[Stage 120:==================================================>(1192 + 8) / 1200]

+-------------------+--------+
|classificacao_final|   count|
+-------------------+--------+
|               NULL|11299279|
|                  1| 3436664|
|                  2| 2163258|
|                  3| 1387603|
|               1.01|  214745|
|               2.01|  134358|
|               1.02|   89814|
|               1.03|   53312|
|               3.01|   52079|
|               2.02|   38293|
|               2.03|   24376|
|               3.02|   16592|
|               3.03|   15328|
+-------------------+--------+



In [24]:
# ============================================================================
# VALIDAÇÃO - PREPARAR DADOS PARA EQUAÇÃO CONTÁBIL
# ============================================================================

print("⚖️ Preparando validação da equação contábil...")
print("   Carregando saldos...")

# Carregar saldos das contas
df_saldos = spark.sql(f"""
    SELECT 
        s.id_ecd,
        s.dt_referencia,
        s.cd_conta_analitica,
        s.vl_saldo_final,
        s.ind_st_saldo_final
    FROM {TABLE_SALDOS} s
    INNER JOIN (
        SELECT id_ecd, dt_referencia, MAX(id_sal_per_ident_periodo) as max_periodo
        FROM {TABLE_SALDOS}
        WHERE dt_referencia >= {ANO_REFERENCIA}
        GROUP BY id_ecd, dt_referencia
    ) ultimo ON s.id_ecd = ultimo.id_ecd 
        AND s.dt_referencia = ultimo.dt_referencia
        AND s.id_sal_per_ident_periodo = ultimo.max_periodo
""")

print(f"✅ {df_saldos.count():,} saldos carregados")

# Join classificação com saldos
df_validacao = df_classificado.join(
    df_saldos,
    (df_classificado.id_ecd == df_saldos.id_ecd) &
    (df_classificado.dt_referencia == df_saldos.dt_referencia) &
    (df_classificado.cd_conta_anl == df_saldos.cd_conta_analitica),
    'inner'
).select(
    df_classificado['*'],
    df_saldos.vl_saldo_final,
    df_saldos.ind_st_saldo_final
)

# Converter saldo para valor com sinal
df_validacao = df_validacao.withColumn('vl_saldo_sinalizado',
    when(col('ind_st_saldo_final') == 'D', col('vl_saldo_final'))
    .otherwise(-col('vl_saldo_final'))
)

print("✅ Dados preparados para validação")
df_validacao.select('id_ecd', 'cd_conta_anl', 'classificacao_final', 'vl_saldo_sinalizado').show(10)

⚖️ Preparando validação da equação contábil...
   Carregando saldos...


✅ 451,446,396 saldos carregados
✅ Dados preparados para validação


[Stage 150:>                                                        (0 + 1) / 1]]]

+------+------------+-------------------+-------------------+
|id_ecd|cd_conta_anl|classificacao_final|vl_saldo_sinalizado|
+------+------------+-------------------+-------------------+
|346788|         475|               NULL|               0.00|
|346804|         179|                  1|          -18085.96|
|346811|         128|                  1|          -37926.19|
|346811|         356|                  3|               0.00|
|346820|        1023|               1.02|         -211152.72|
|346820|        1067|                  1|            -250.00|
|346820|         129|                  1|         -160000.00|
|346820|         531|               NULL|         2612409.48|
|346821|          83|               NULL|           -1067.27|
|346828|   120300004|                  1|            2060.00|
+------+------------+-------------------+-------------------+
only showing top 10 rows



In [25]:
# ============================================================================
# VALIDAÇÃO - CALCULAR EQUAÇÃO CONTÁBIL (ATIVO - PASSIVO - PL = 0)
# ============================================================================

print("⚖️ Calculando equação contábil por empresa...")

# Agregar por empresa e grupo
df_agregado = df_validacao.groupBy(
    'id_ecd',
    'dt_referencia',
    'classificacao_nivel_1'
).agg(
    sum('vl_saldo_sinalizado').alias('total_grupo')
)

# Pivotar
df_eq_contabil = df_agregado.groupBy('id_ecd', 'dt_referencia').pivot(
    'classificacao_nivel_1',
    ['1', '2', '3']
).agg(
    first('total_grupo')
).fillna(0).withColumnRenamed('1', 'total_ativo') \
          .withColumnRenamed('2', 'total_passivo') \
          .withColumnRenamed('3', 'total_resultado')

# Calcular diferença (ATIVO + PASSIVO should be ~0)
df_eq_contabil = df_eq_contabil.withColumn(
    'diferenca_absoluta',
    abs(col('total_ativo') + col('total_passivo'))
).withColumn(
    'validacao_ok',
    when(col('diferenca_absoluta') <= TOLERANCIA_EQUACAO_CONTABIL, 1).otherwise(0)
).withColumn(
    'percentual_diferenca',
    when(col('total_ativo') != 0, (col('diferenca_absoluta') / abs(col('total_ativo'))) * 100)
    .otherwise(None)
)

total_empresas = df_eq_contabil.count()
empresas_ok = df_eq_contabil.filter(col('validacao_ok') == 1).count()

print(f"\n✅ Validação concluída:")
print(f"   Total de empresas: {total_empresas:,}")
print(f"   Empresas OK: {empresas_ok:,} ({empresas_ok/total_empresas*100:.1f}%)")
print(f"   Com divergência: {total_empresas - empresas_ok:,} ({(total_empresas-empresas_ok)/total_empresas*100:.1f}%)")

print("\n⚠️ Top 10 empresas com maior divergência:")
df_eq_contabil.orderBy(desc('diferenca_absoluta')).select(
    'id_ecd', 'total_ativo', 'total_passivo', 'diferenca_absoluta', 'percentual_diferenca'
).show(10, truncate=False)

⚖️ Calculando equação contábil por empresa...


                                                                                7]


✅ Validação concluída:
   Total de empresas: 29,165
   Empresas OK: 6,441 (22.1%)
   Com divergência: 22,724 (77.9%)

⚠️ Top 10 empresas com maior divergência:


[Stage 234:=====================================================> (28 + 1) / 29]7]

+------+---------------+-----------------+------------------+--------------------+
|id_ecd|total_ativo    |total_passivo    |diferenca_absoluta|percentual_diferenca|
+------+---------------+-----------------+------------------+--------------------+
|389051|604198751260.77|-19492378242.78  |584706373017.99   |96.773847           |
|487551|832306377211.34|-1066234519644.39|233928142433.05   |28.106014           |
|432339|42846301250.68 |-9567635196.00   |33278666054.68    |77.669869           |
|486553|5518077408.18  |21260711787.60   |26778789195.78    |485.292018          |
|469377|25720897907.84 |-815676381.44    |24905221526.40    |96.828741           |
|432739|14837028548.57 |-7383700739.00   |7453327809.57     |50.234640           |
|405311|5895110940.69  |0.00             |5895110940.69     |100.000000          |
|420245|6973926556.80  |-1822206188.21   |5151720368.59     |73.871159           |
|459819|281177890.10   |4209342652.59    |4490520542.69     |1597.038992         |
|386

In [26]:
# ============================================================================
# ANÁLISE DE OUTLIERS - CONTAS NÃO CLASSIFICADAS
# ============================================================================

print("🔍 Identificando contas não classificadas (outliers)...")

# Contas sem classificação
df_outliers = df_classificado.filter(
    col('classificacao_nivel_1').isNull()
)

total_outliers = df_outliers.count()
total = df_classificado.count()

print(f"\n⚠️ Outliers: {total_outliers:,} / {total:,} ({total_outliers/total*100:.1f}%)")

# Análise por padrão estrutural
print("\n--- Outliers por Padrão Estrutural ---")
df_outliers.groupBy('padrao_estrutural', 'cd_natureza').count() \
    .orderBy(desc('count')).show(15)

# Amostra de outliers
print("\n--- Amostra de Outliers ---")
df_outliers.select('cd_conta_anl', 'nm_conta_anl', 'cd_natureza', 'nivel', 'padrao_estrutural') \
    .show(20, truncate=False)

# Outliers por empresa
print("\n--- Empresas com Mais Outliers ---")
df_outliers.groupBy('id_ecd', 'dt_referencia').count() \
    .withColumnRenamed('count', 'qtd_outliers') \
    .orderBy(desc('qtd_outliers')).show(10)

🔍 Identificando contas não classificadas (outliers)...



⚠️ Outliers: 11,299,279 / 18,925,701 (59.7%)

--- Outliers por Padrão Estrutural ---


+-----------------+-----------+-------+
|padrao_estrutural|cd_natureza|  count|
+-----------------+-----------+-------+
|       PREFIXO_BP|          D|3534070|
|  SOMENTE_NUMEROS|          D|2873818|
|       PREFIXO_BP|          C|1410623|
|  SOMENTE_NUMEROS|          C|1303087|
|      PREFIXO_DRE|          D| 784666|
|            OUTRO|          C| 356446|
|            OUTRO|          D| 333292|
|      PREFIXO_DRE|          C| 303062|
|    PADRAO_PONTOS|          D| 302961|
|    PADRAO_PONTOS|          C|  97254|
+-----------------+-----------+-------+


--- Amostra de Outliers ---
+--------------+-----------------------------------------------+-----------+-----+-----------------+
|cd_conta_anl  |nm_conta_anl                                   |cd_natureza|nivel|padrao_estrutural|
+--------------+-----------------------------------------------+-----------+-----+-----------------+
|480           |(-) SIMPLES NACIONAL                           |C          |14   |SOMENTE_NUMEROS  |
|55   

[Stage 261:==================================================>(1198 + 2) / 1200]

+------+-------------+------------+
|id_ecd|dt_referencia|qtd_outliers|
+------+-------------+------------+
|432550|       202401|      434099|
|487536|       202401|      330477|
|487521|       202401|      330474|
|392704|       202401|      221321|
|389652|       202401|      103213|
|434651|       202401|       85878|
|389665|       202401|       72929|
|486805|       202401|       64792|
|431352|       202401|       57155|
|389409|       202401|       55450|
+------+-------------+------------+
only showing top 10 rows



In [27]:
# ============================================================================
# ANÁLISE DE OUTLIERS - BAIXA CONFIANÇA
# ============================================================================

print("🔍 Identificando contas com baixa confiança...")

MIN_CONFIANCA = 0.70

# Contas classificadas mas com baixa confiança
df_baixa_confianca = df_classificado.filter(
    (col('classificacao_nivel_1').isNotNull()) &
    (col('confianca_nivel_1') < MIN_CONFIANCA)
)

total_baixa = df_baixa_confianca.count()
print(f"\n⚠️ Contas com confiança < {MIN_CONFIANCA}: {total_baixa:,} ({total_baixa/total*100:.1f}%)")

# Análise por grupo
print("\n--- Baixa Confiança por Grupo ---")
df_baixa_confianca.groupBy('classificacao_nivel_1_desc', 'confianca_nivel_1').count() \
    .orderBy(desc('count')).show()

# Combinar: Outliers completos (não classificados + baixa confiança)
df_outliers_completo = df_classificado.filter(
    col('classificacao_nivel_1').isNull() | 
    (col('confianca_nivel_1') < MIN_CONFIANCA)
)

total_outliers_completo = df_outliers_completo.count()
print(f"\n📊 Total de Outliers (não class. + baixa conf.): {total_outliers_completo:,} ({total_outliers_completo/total*100:.1f}%)")

🔍 Identificando contas com baixa confiança...



⚠️ Contas com confiança < 0.7: 226,896 (1.2%)

--- Baixa Confiança por Grupo ---


+--------------------------+-----------------+------+
|classificacao_nivel_1_desc|confianca_nivel_1| count|
+--------------------------+-----------------+------+
|                     ATIVO|              0.0|109726|
|                 RESULTADO|              0.0| 87296|
|                   PASSIVO|              0.0| 29874|
+--------------------------+-----------------+------+



[Stage 270:==============================================>   (1121 + 28) / 1200]


📊 Total de Outliers (não class. + baixa conf.): 11,526,175 (60.9%)


In [28]:
# ============================================================================
# ANÁLISE DE COBERTURA POR NÍVEL HIERÁRQUICO
# ============================================================================

print("📊 Analisando cobertura por nível hierárquico...")

df_cobertura = df_classificado.groupBy('nivel').agg(
    count('*').alias('total_contas'),
    sum(when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0)).alias('classificadas_n1'),
    sum(when(col('classificacao_nivel_2') != col('classificacao_nivel_1'), 1).otherwise(0)).alias('classificadas_n2'),
    sum(when(col('classificacao_nivel_3') != col('classificacao_nivel_2'), 1).otherwise(0)).alias('classificadas_n3'),
    avg('confianca_nivel_1').alias('confianca_media')
).withColumn(
    'percentual_n1',
    (col('classificadas_n1') / col('total_contas')) * 100
).withColumn(
    'percentual_n2',
    (col('classificadas_n2') / col('classificadas_n1')) * 100
).withColumn(
    'percentual_n3',
    (col('classificadas_n3') / col('classificadas_n2')) * 100
).orderBy('nivel')

print("\n--- Cobertura por Nível Hierárquico ---")
df_cobertura.show(15, truncate=False)

# Salvar para análise
cobertura_pandas = df_cobertura.toPandas()
print("\n📈 Resumo:")
print(f"   Níveis com melhor cobertura: {cobertura_pandas.nlargest(3, 'percentual_n1')[['nivel', 'percentual_n1']].to_string(index=False)}")

📊 Analisando cobertura por nível hierárquico...

--- Cobertura por Nível Hierárquico ---


+-----+------------+----------------+----------------+----------------+-------------------+------------------+------------------+-------------+
|nivel|total_contas|classificadas_n1|classificadas_n2|classificadas_n3|confianca_media    |percentual_n1     |percentual_n2     |percentual_n3|
+-----+------------+----------------+----------------+----------------+-------------------+------------------+------------------+-------------+
|1    |179342      |103654          |1164            |0               |0.45519287171995787|57.79683509718861 |1.122966793370251 |0.0          |
|2    |425382      |213256          |12251           |0               |0.36076373706456333|50.132821793117714|5.744738717785197 |0.0          |
|3    |1022870     |575179          |31632           |0               |0.40377995248661586|56.23187697361346 |5.499505371371347 |0.0          |
|4    |1856900     |982799          |68031           |0               |0.39745382088430087|52.92686735957779 |6.922168215474375 |0.0    

[Stage 276:==================================================>(1198 + 2) / 1200]


📈 Resumo:
   Níveis com melhor cobertura:  nivel  percentual_n1
    37          100.0
    38          100.0
    39          100.0


In [29]:
# ============================================================================
# ESTATÍSTICAS POR ESTADO (UF)
# ============================================================================

print("🗺️ Analisando distribuição por Estado...")

# Join com informações cadastrais
df_empresas_info = spark.sql(f"""
    SELECT id_ecd, dt_referencia, nm_empresarial, nu_cnpj, cd_uf
    FROM {TABLE_IDENTIFICACAO}
    WHERE dt_referencia >= {ANO_REFERENCIA}
""")

df_por_uf = df_classificado.join(
    df_empresas_info,
    on=['id_ecd', 'dt_referencia'],
    how='inner'
)

print("\n--- Empresas por Estado ---")
df_por_uf.select('cd_uf', 'id_ecd').distinct() \
    .groupBy('cd_uf').count() \
    .withColumnRenamed('count', 'qtd_empresas') \
    .orderBy(desc('qtd_empresas')).show(10)

print("\n--- Contas por Estado ---")
df_por_uf.groupBy('cd_uf').agg(
    count('*').alias('total_contas'),
    sum(when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0)).alias('classificadas'),
    avg('confianca_nivel_1').alias('confianca_media')
).withColumn(
    'percentual_class',
    (col('classificadas') / col('total_contas')) * 100
).orderBy(desc('total_contas')).show(10)

🗺️ Analisando distribuição por Estado...

--- Empresas por Estado ---


+-----+------------+
|cd_uf|qtd_empresas|
+-----+------------+
|   SC|       27444|
|   SP|        2953|
|   PR|        1433|
|   RS|         952|
|   MG|         320|
|   RJ|         237|
|   ES|         139|
|   GO|         109|
|   MT|          76|
|   MS|          56|
+-----+------------+
only showing top 10 rows


--- Contas por Estado ---


[Stage 308:===========================================>           (23 + 4) / 29]

+-----+------------+-------------+-------------------+------------------+
|cd_uf|total_contas|classificadas|    confianca_media|  percentual_class|
+-----+------------+-------------+-------------------+------------------+
|   SC|     8218480|      2742689| 0.2572047385895463| 33.37221724698484|
|   SP|     1330899|       642963|0.37452458826689433| 48.31042776348919|
|   PR|      751764|       370184|0.38618609031565376| 49.24204936655653|
|   RS|      687509|       351735|0.40521360447641247| 51.16078480427165|
|   MG|      160676|        84410| 0.4190793895789746| 52.53429261370709|
|   GO|       82037|        26653| 0.2590440898618897| 32.48899886636518|
|   RJ|       71362|        36779| 0.4051119643507769|51.538634006894426|
|   MT|       41196|        28493| 0.5498397902709061| 69.16448198854258|
|   MS|       39508|        18419| 0.3562417738179917|46.620937531639164|
|   DF|       39469|         6014|0.11604043679849979|15.237274823279028|
+-----+------------+-------------+----

In [30]:
# ============================================================================
# ANÁLISE TEMPORAL - EVOLUÇÃO POR ANO
# ============================================================================

print("📅 Analisando distribuição temporal...")

# Extrair ano da dt_referencia
df_temporal = df_classificado.withColumn(
    'ano',
    substring(col('dt_referencia').cast('string'), 1, 4).cast('int')
)

print("\n--- Distribuição por Ano ---")
df_temporal.groupBy('ano').agg(
    countDistinct('id_ecd').alias('qtd_empresas'),
    count('*').alias('total_contas'),
    sum(when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0)).alias('classificadas'),
    avg('confianca_nivel_1').alias('confianca_media')
).withColumn(
    'percentual_class',
    (col('classificadas') / col('total_contas')) * 100
).orderBy('ano').show()

print("\n--- Distribuição por Grupo e Ano ---")
df_temporal.groupBy('ano', 'classificacao_nivel_1_desc').count() \
    .orderBy('ano', desc('count')).show(20)

📅 Analisando distribuição temporal...

--- Distribuição por Ano ---


+----+------------+------------+-------------+------------------+-----------------+
| ano|qtd_empresas|total_contas|classificadas|   confianca_media| percentual_class|
+----+------------+------------+-------------+------------------+-----------------+
|2024|       59284|    18925701|      7626422|0.3127821157060444|40.29664211645317|
+----+------------+------------+-------------+------------------+-----------------+


--- Distribuição por Grupo e Ano ---


[Stage 319:================================================> (1166 + 28) / 1200]

+----+--------------------------+--------+
| ano|classificacao_nivel_1_desc|   count|
+----+--------------------------+--------+
|2024|                      NULL|11299279|
|2024|                     ATIVO| 3794535|
|2024|                   PASSIVO| 2360285|
|2024|                 RESULTADO| 1471602|
+----+--------------------------+--------+



In [31]:
# ============================================================================
# RESUMO EXECUTIVO DA CLASSIFICAÇÃO - VERSÃO OTIMIZADA
# ============================================================================

print("=" * 80)
print("RESUMO EXECUTIVO - CLASSIFICAÇÃO DE CONTAS ECD")
print("=" * 80)

# Calcular métricas em uma única passada
print("\n⏳ Calculando métricas gerais...")

metricas = df_classificado.agg(
    count('*').alias('total_contas'),
    countDistinct('id_ecd').alias('total_empresas'),
    sum(when(col('classificacao_nivel_1').isNotNull(), 1).otherwise(0)).alias('class_n1'),
    sum(when((col('classificacao_nivel_2').isNotNull()) & 
             (col('classificacao_nivel_2') != col('classificacao_nivel_1')), 1).otherwise(0)).alias('class_n2'),
    sum(when((col('classificacao_nivel_3').isNotNull()) & 
             (col('classificacao_nivel_3') != col('classificacao_nivel_2')), 1).otherwise(0)).alias('class_n3'),
    sum(when(col('confianca_nivel_1') > 0, col('confianca_nivel_1')).otherwise(0)).alias('soma_confianca'),
    sum(when(col('confianca_nivel_1') > 0, 1).otherwise(0)).alias('total_com_confianca'),
    sum(when(col('classificacao_nivel_1').isNull(), 1).otherwise(0)).alias('outliers')
).collect()[0]

total_contas = metricas['total_contas']
total_empresas = metricas['total_empresas']
class_n1 = metricas['class_n1']
class_n2 = metricas['class_n2']
class_n3 = metricas['class_n3']
conf_media = metricas['soma_confianca'] / metricas['total_com_confianca'] if metricas['total_com_confianca'] > 0 else 0
outliers_total = metricas['outliers']

print(f"\n📊 VOLUME DE DADOS")
print(f"   Total de empresas: {total_empresas:,}")
print(f"   Total de contas: {total_contas:,}")
print(f"   Média contas/empresa: {total_contas/total_empresas:,.0f}")

print(f"\n🎯 COBERTURA DE CLASSIFICAÇÃO")
print(f"   Nível 1 (grupos): {class_n1:,} ({class_n1/total_contas*100:.1f}%)")
print(f"   Nível 2 (subgrupos): {class_n2:,} ({class_n2/class_n1*100:.1f}% dos N1)" if class_n1 > 0 else "   Nível 2 (subgrupos): 0")
print(f"   Nível 3 (detalhado): {class_n3:,} ({class_n3/class_n2*100:.1f}% dos N2)" if class_n2 > 0 else "   Nível 3 (detalhado): 0")

print(f"\n💯 CONFIANÇA MÉDIA")
print(f"   Confiança geral: {conf_media:.2%}")

print(f"\n⚠️ OUTLIERS")
print(f"   Contas não classificadas: {outliers_total:,} ({outliers_total/total_contas*100:.1f}%)")

# Distribuição por grupo (apenas o essencial)
print(f"\n📈 DISTRIBUIÇÃO POR GRUPO")
df_classificado.groupBy('classificacao_nivel_1_desc').count() \
    .orderBy(desc('count')).show(truncate=False)

# Validação (usar df já calculado)
print(f"\n⚖️ VALIDAÇÃO EQUAÇÃO CONTÁBIL")
valid_stats = df_eq_contabil.agg(
    count('*').alias('total'),
    sum(when(col('validacao_ok') == 1, 1).otherwise(0)).alias('ok')
).collect()[0]

total_validacao = valid_stats['total']
empresas_validadas = valid_stats['ok']
print(f"   Empresas validadas: {empresas_validadas:,} / {total_validacao:,} ({empresas_validadas/total_validacao*100:.1f}%)")

print("\n" + "=" * 80)

RESUMO EXECUTIVO - CLASSIFICAÇÃO DE CONTAS ECD

⏳ Calculando métricas gerais...



📊 VOLUME DE DADOS
   Total de empresas: 59,284
   Total de contas: 18,925,701
   Média contas/empresa: 319

🎯 COBERTURA DE CLASSIFICAÇÃO
   Nível 1 (grupos): 7,626,422 (40.3%)
   Nível 2 (subgrupos): 638,897 (8.4% dos N1)
   Nível 3 (detalhado): 0 (0.0% dos N2)

💯 CONFIANÇA MÉDIA
   Confiança geral: 80.00%

⚠️ OUTLIERS
   Contas não classificadas: 11,299,279 (59.7%)

📈 DISTRIBUIÇÃO POR GRUPO


+--------------------------+--------+
|classificacao_nivel_1_desc|count   |
+--------------------------+--------+
|NULL                      |11299279|
|ATIVO                     |3794535 |
|PASSIVO                   |2360285 |
|RESULTADO                 |1471602 |
+--------------------------+--------+


⚖️ VALIDAÇÃO EQUAÇÃO CONTÁBIL


[Stage 345:=====================================================> (28 + 1) / 29]7]

   Empresas validadas: 6,441 / 29,165 (22.1%)



In [32]:
# ============================================================================
# ANÁLISE DE QUALIDADE - DISTRIBUIÇÃO DETALHADA
# ============================================================================

print("🔬 Análise detalhada da qualidade das classificações...\n")

# Top 30 classificações mais frequentes
print("--- Top 30 Classificações Finais ---")
df_classificado.groupBy('classificacao_final', 'classificacao_nivel_1_desc').count() \
    .orderBy(desc('count')).show(30, truncate=False)

# Análise de confiança por classificação
print("\n--- Confiança Média por Classificação (Top 20) ---")
df_classificado.filter(col('classificacao_nivel_1').isNotNull()) \
    .groupBy('classificacao_final').agg(
        count('*').alias('total'),
        avg('confianca_nivel_1').alias('confianca_media')
    ).orderBy(desc('total')).show(20)

# Distribuição de padrões estruturais por grupo
print("\n--- Padrões Estruturais por Grupo ---")
df_classificado.groupBy('classificacao_nivel_1_desc', 'padrao_estrutural').count() \
    .orderBy('classificacao_nivel_1_desc', desc('count')).show(30, truncate=False)

🔬 Análise detalhada da qualidade das classificações...

--- Top 30 Classificações Finais ---


+-------------------+--------------------------+--------+
|classificacao_final|classificacao_nivel_1_desc|count   |
+-------------------+--------------------------+--------+
|NULL               |NULL                      |11299279|
|1                  |ATIVO                     |3436664 |
|2                  |PASSIVO                   |2163258 |
|3                  |RESULTADO                 |1387603 |
|1.01               |ATIVO                     |214745  |
|2.01               |PASSIVO                   |134358  |
|1.02               |ATIVO                     |89814   |
|1.03               |ATIVO                     |53312   |
|3.01               |RESULTADO                 |52079   |
|2.02               |PASSIVO                   |38293   |
|2.03               |PASSIVO                   |24376   |
|3.02               |RESULTADO                 |16592   |
|3.03               |RESULTADO                 |15328   |
+-------------------+--------------------------+--------+


--- Confianç

+-------------------+-------+------------------+
|classificacao_final|  total|   confianca_media|
+-------------------+-------+------------------+
|                  1|3436664|0.7744871189034868|
|                  2|2163258|0.7892203334046142|
|                  3|1387603|0.7515048612608417|
|               1.01| 214745| 0.799821183263857|
|               2.01| 134358|0.7997856473005011|
|               1.02|  89814|0.7994922840537175|
|               1.03|  53312|0.7996698679471824|
|               3.01|  52079|0.7719195837093674|
|               2.02|  38293|0.7994777113310533|
|               2.03|  24376|0.7782080735149316|
|               3.02|  16592|0.7725650916104136|
|               3.03|  15328|0.7590814196242164|
+-------------------+-------+------------------+


--- Padrões Estruturais por Grupo ---


[Stage 376:================================================> (1173 + 27) / 1200]

+--------------------------+-----------------+-------+
|classificacao_nivel_1_desc|padrao_estrutural|count  |
+--------------------------+-----------------+-------+
|NULL                      |PREFIXO_BP       |4944693|
|NULL                      |SOMENTE_NUMEROS  |4176905|
|NULL                      |PREFIXO_DRE      |1087728|
|NULL                      |OUTRO            |689738 |
|NULL                      |PADRAO_PONTOS    |400215 |
|ATIVO                     |SOMENTE_NUMEROS  |2777090|
|ATIVO                     |PADRAO_PONTOS    |918628 |
|ATIVO                     |OUTRO            |98817  |
|PASSIVO                   |SOMENTE_NUMEROS  |1411361|
|PASSIVO                   |PADRAO_PONTOS    |875802 |
|PASSIVO                   |OUTRO            |73122  |
|RESULTADO                 |SOMENTE_NUMEROS  |1253312|
|RESULTADO                 |PADRAO_PONTOS    |150453 |
|RESULTADO                 |OUTRO            |67837  |
+--------------------------+-----------------+-------+



In [33]:
# ============================================================================
# ANÁLISE DE PADRÕES - CONTAS MAIS COMUNS POR CLASSIFICAÇÃO
# ============================================================================

print("🔍 Identificando contas mais comuns por classificação...\n")

# Top contas no Ativo Circulante
print("--- Top 15 Contas no ATIVO CIRCULANTE (1.01) ---")
df_classificado.filter(col('classificacao_nivel_2') == '1.01') \
    .groupBy('cd_conta_anl', 'nm_conta_anl', 'classificacao_nivel_3').count() \
    .orderBy(desc('count')).show(15, truncate=False)

# Top contas no Passivo Circulante
print("\n--- Top 15 Contas no PASSIVO CIRCULANTE (2.01) ---")
df_classificado.filter(col('classificacao_nivel_2') == '2.01') \
    .groupBy('cd_conta_anl', 'nm_conta_anl', 'classificacao_nivel_3').count() \
    .orderBy(desc('count')).show(15, truncate=False)

# Top contas no Patrimônio Líquido
print("\n--- Top 15 Contas no PATRIMÔNIO LÍQUIDO (2.03) ---")
df_classificado.filter(col('classificacao_nivel_2') == '2.03') \
    .groupBy('cd_conta_anl', 'nm_conta_anl', 'classificacao_nivel_3').count() \
    .orderBy(desc('count')).show(15, truncate=False)

# Top contas na DRE
print("\n--- Top 15 Contas na DRE (3.x) ---")
df_classificado.filter(col('classificacao_nivel_1') == '3') \
    .groupBy('cd_conta_anl', 'nm_conta_anl', 'classificacao_nivel_3').count() \
    .orderBy(desc('count')).show(15, truncate=False)

🔍 Identificando contas mais comuns por classificação...

--- Top 15 Contas no ATIVO CIRCULANTE (1.01) ---


+--------------------------------+--------------------------------+---------------------+-----+
|cd_conta_anl                    |nm_conta_anl                    |classificacao_nivel_3|count|
+--------------------------------+--------------------------------+---------------------+-----+
|1.01                            |ATIVO CIRCULANTE                |1.01                 |533  |
|101                             |OUTROS INVESTIMENTOS PERMANENTES|1.01                 |498  |
|1.01                            |CIRCULANTE                      |1.01                 |340  |
|1.01.01                         |CAIXA E EQUIVALENTES DE CAIXA   |1.01                 |306  |
|10100000000000000000000002      |ATIVO CIRCULANTE                |1.01                 |249  |
|1.01.01.02.01.01.01.01.01.01.011|Banco SICOOB                    |1.01                 |237  |
|1.01.01.02.01.01.01.01.01.01.001|Banco do Brasil SA              |1.01                 |229  |
|1.01.03                         |ESTOQU

+--------------------------------+-----------------------------------+---------------------+-----+
|cd_conta_anl                    |nm_conta_anl                       |classificacao_nivel_3|count|
+--------------------------------+-----------------------------------+---------------------+-----+
|201                             |ADIANTAMENTOS A CLIENTES           |2.01                 |1472 |
|201                             |ADIANTAMENTOS DE CLIENTES          |2.01                 |1355 |
|2.01                            |PASSIVO CIRCULANTE                 |2.01                 |558  |
|201                             |ESTOQUES                           |2.01                 |508  |
|2.01                            |CIRCULANTE                         |2.01                 |308  |
|2.01.01.01.01.01.01.01.01.01.001|Fornecedores                       |2.01                 |258  |
|20100000000000000000000002      |PASSIVO CIRCULANTE                 |2.01                 |254  |
|2.01.05.0

+--------------------------------+----------------------------------------+---------------------+-----+
|cd_conta_anl                    |nm_conta_anl                            |classificacao_nivel_3|count|
+--------------------------------+----------------------------------------+---------------------+-----+
|2.03.10.01.01.01.01.01.01.01.002|() Prejuizos Acumulados                 |2.03                 |293  |
|2.03.01                         |CAPITAL SOCIAL                          |2.03                 |279  |
|2.03                            |PATRIMÔNIO LÍQUIDO                      |2.03                 |189  |
|2.03                            |PATRIMONIO LIQUIDO                      |2.03                 |153  |
|203                             |ENERGIA ELÉTRICA, ÁGUA E TELEFONE A PAGA|2.03                 |148  |
|2030                            |OBRIGAÇÕES TRABALHISTAS E SOCIAIS       |2.03                 |141  |
|2.03.02                         |RESERVAS                      

[Stage 388:==================================================>(1197 + 3) / 1200]

+------------+-------------------------------------------+---------------------+-----+
|cd_conta_anl|nm_conta_anl                               |classificacao_nivel_3|count|
+------------+-------------------------------------------+---------------------+-----+
|3           |DISPONÍVEL                                 |3                    |9376 |
|030         |(-) CUSTO DOS PRODUTOS/MERCADORIAS/SERVICOS|3                    |6080 |
|30          |ICMS A RECUPERAR                           |3                    |5797 |
|336         |INSS                                       |3                    |5690 |
|382         |EMPRÉSTIMOS E FINANCIAMENTOS               |3                    |5091 |
|332         |PRÓ-LABORE                                 |3                    |4947 |
|362         |SERVIÇOS PRESTADOS POR TERCEIROS           |3                    |4690 |
|350         |TAXAS DIVERSAS                             |3                    |4671 |
|337         |FGTS                         